In [13]:
from functions import *
from ipynb.fs.full.Student_Info_and_Registration import student_info_reg
from ipynb.fs.full.Assessments import assessments
from ipynb.fs.full.Student_Assessment import student_assessment

---

# Assessments and Student Assessments

---

## Assessments and Student Assessments Merged Dataframe:

In [15]:
assessments

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


In [16]:
student_assessment

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78
1,1752,28400,22,0,70
2,1752,31604,17,0,72
3,1752,32885,26,0,69
4,1752,38053,19,0,79
...,...,...,...,...,...
173907,37443,527538,227,0,60
173908,37443,534672,229,0,100
173909,37443,546286,215,0,80
173910,37443,546724,230,0,100


Here we will merge the assessments and student assessments dataframes in order to combine our student scores and submission dates with assessment type, date of the assessment, and weight of the assessment.

In [17]:
# merges dataframes student_assessment with assessments with a full outer join on their common ID id_assessment
# creates a column _merge which tells you if the id_assessment was found in one or both dataframes
merged_assessments = student_assessment.merge(assessments, how='outer', on=['id_assessment'] ,indicator=True)
merged_assessments.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,_merge
0,1752,11391,18,0,78,AAA,2013J,TMA,19.0,10.0,both
1,1752,28400,22,0,70,AAA,2013J,TMA,19.0,10.0,both
2,1752,31604,17,0,72,AAA,2013J,TMA,19.0,10.0,both
3,1752,32885,26,0,69,AAA,2013J,TMA,19.0,10.0,both
4,1752,38053,19,0,79,AAA,2013J,TMA,19.0,10.0,both


Our new merge column tells us if the data maps perfectly, or if it is only found on the right or left side, the right side being the assessments dataframe and the left side being the student_assessments dataframe. 

**Rows that do not map:**

In [18]:
missing_exams = merged_assessments.loc[merged_assessments['_merge'] != 'both'].reset_index(drop=True)
missing_exams

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,_merge
0,1757,<NA>,<NA>,<NA>,<NA>,AAA,2013J,Exam,268.0,100.0,right_only
1,1763,<NA>,<NA>,<NA>,<NA>,AAA,2014J,Exam,269.0,100.0,right_only
2,14990,<NA>,<NA>,<NA>,<NA>,BBB,2013B,Exam,240.0,100.0,right_only
3,15002,<NA>,<NA>,<NA>,<NA>,BBB,2013J,Exam,268.0,100.0,right_only
4,15014,<NA>,<NA>,<NA>,<NA>,BBB,2014B,Exam,234.0,100.0,right_only
5,15025,<NA>,<NA>,<NA>,<NA>,BBB,2014J,Exam,262.0,100.0,right_only
6,40087,<NA>,<NA>,<NA>,<NA>,CCC,2014B,Exam,241.0,100.0,right_only
7,40088,<NA>,<NA>,<NA>,<NA>,CCC,2014J,Exam,269.0,100.0,right_only
8,30713,<NA>,<NA>,<NA>,<NA>,EEE,2013J,Exam,235.0,100.0,right_only
9,30718,<NA>,<NA>,<NA>,<NA>,EEE,2014B,Exam,228.0,100.0,right_only


In [4]:
missing_exams_list = list(missing_exams['id_assessment'])

In [5]:
md(f'''
These {len(missing_exams)} rows all have entries in the assessments dataframe but have no match in the student_assessment dataframe. This indicates that no students in our data took these exams, and so we will drop them, and then the merge column since it will have no more useful information.
''')


These 18 rows all have entries in the assessments dataframe but have no match in the student_assessment dataframe. This indicates that no students in our data took these exams, and so we will drop them, and then the merge column since it will have no more useful information.


In [6]:
# remove tests that students did not take
merged_assessments = merged_assessments.dropna(subset=['id_student'])
# drop the merge column since it is no longer of use
# reset the index to be consecutive again
merged_assessments = merged_assessments.drop(columns=['_merge']).reset_index(drop=True)
# order the columns
merged_assessments = merged_assessments[['code_module', 'code_presentation', 'id_student', 'id_assessment', 'assessment_type', 'date_submitted', 'date', 'weight', 'score']]

**Removing Eliminated Students**

**Merged Assessment/Student_info dataframes**

We will be predicting student assessment scores based on their demographics so we need a dataframe that combines student info with our merged assessments

In [7]:
# merged 'student info/assessments' with a full outer join on their common columns
merged_si_assm = merged_assessments.merge(student_info_reg, how='outer', on=['id_student', 'code_module', 'code_presentation'], indicator=True)
merged_si_assm.head()

,code_module,code_presentation,id_student,id_assessment,assessment_type,date_submitted,date,weight,score,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration,_merge
0,AAA,2013J,11391,1752,TMA,18,19.0,10.0,78,East Anglian Region,90-100%,55<=,M,HE Qualification,False,Pass,-159,<NA>,both
1,AAA,2013J,11391,1753,TMA,53,54.0,20.0,85,East Anglian Region,90-100%,55<=,M,HE Qualification,False,Pass,-159,<NA>,both
2,AAA,2013J,11391,1754,TMA,115,117.0,20.0,80,East Anglian Region,90-100%,55<=,M,HE Qualification,False,Pass,-159,<NA>,both
3,AAA,2013J,11391,1755,TMA,164,166.0,20.0,85,East Anglian Region,90-100%,55<=,M,HE Qualification,False,Pass,-159,<NA>,both
4,AAA,2013J,11391,1756,TMA,212,215.0,30.0,82,East Anglian Region,90-100%,55<=,M,HE Qualification,False,Pass,-159,<NA>,both


In [28]:
stud_assm_uniques = student_assessment['id_student'].nunique()
stud_info_uniques = student_info['id_student'].nunique()
unique_diff = stud_info_uniques - stud_assm_uniques
md(f'''
We found earlier in student assessments that there were only {"{:,}".format(stud_assm_uniques)} unique students with test scores
out of a total of {"{:,}".format(stud_info_uniques)} students in student info.
This means that there are {unique_diff} students without assessment scores which should be found by locating all of the data with a value of `right_only`
in the `_merge` column.
''')


We found earlier in student assessments that there were only 23,369 unique students with test scores
out of a total of 28,785 students in student info.
This means that there are 5416 students without assessment scores which should be found by locating all of the data with a value of `right_only`
in the `_merge` column.


**Students without any test scores**:

In [31]:
# variable for where merge is right_only, and only found in student_info
only_student_info = merged_si_assm.loc[merged_si_assm['_merge']=='right_only']
only_student_info

,code_module,code_presentation,id_student,id_assessment,assessment_type,date_submitted,date,weight,score,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration,_merge
173912,AAA,2013J,30268,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,North Western Region,30-40%,35-55,F,A Level or Equivalent,True,Withdrawn,-92,12,right_only
173913,AAA,2013J,135335,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,East Anglian Region,20-30%,0-35,F,Lower Than A Level,False,Withdrawn,-29,30,right_only
173914,AAA,2013J,281589,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,North Western Region,30-40%,0-35,M,HE Qualification,False,Fail,-50,<NA>,right_only
173915,AAA,2013J,292923,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,South East Region,90-100%,35-55,F,A Level or Equivalent,False,Withdrawn,-162,-121,right_only
173916,AAA,2013J,305539,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,Wales,80-90%,0-35,F,Lower Than A Level,False,Withdrawn,-54,-3,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180657,GGG,2014J,2282141,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,Wales,0-10%,35-55,M,A Level or Equivalent,False,Withdrawn,-32,62,right_only
180658,GGG,2014J,2338614,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,Scotland,0-10%,35-55,F,A Level or Equivalent,True,Withdrawn,-23,58,right_only
180659,GGG,2014J,2475886,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,East Anglian Region,40-50%,35-55,F,Lower Than A Level,False,Fail,-31,<NA>,right_only
180660,GGG,2014J,2608143,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,East Midlands Region,60-70%,35-55,M,HE Qualification,False,Withdrawn,-45,48,right_only


In [33]:
md(f'''
    We have {len(only_student_info)} values in only student_info, which means we have students for whom we have no test scores.
    We can drop those which are missing values for the purpose of this dataframe since we are just analyzing test scores
    ''')


    We have 0 values in only assessments, which map to students who had made previous attempts which we eliminated, and 6750 values in only student_info, which means we have students for whom we have no test scores.
    We can drop both of these which are missing values for the purpose of this dataframe since we are just analyzing test scores
    

In [34]:
# removing any student with NaN values in id_assessment or final_result
merged_si_assm = merged_si_assm.dropna(subset=['id_assessment', 'final_result'])

In [36]:
# reordering dataframe columns to group like data
merged_si_assm = merged_si_assm[['code_module', 'code_presentation', 'id_student', 'region', 'imd_band', 'age_band', 'gender', 'highest_education', 'disability', 'final_result', 'id_assessment', 'assessment_type', 'date_submitted', 'date', 'weight', 'score']]

In [37]:
merged_si_assm.dtypes

code_module           object
code_presentation     object
id_student            object
region                string
imd_band              string
age_band              string
gender                string
highest_education     string
disability           boolean
final_result          string
id_assessment         object
assessment_type       string
date_submitted         Int64
date                 Float64
weight               Float64
score                  Int64
dtype: object

In [16]:
# converting the data types back
merged_si_assm = merged_si_assm.astype({'id_assessment': int, 'id_student': int})
merged_si_assm = merged_si_assm.astype({'id_assessment': object, 'id_student': object})

In [17]:
# reset the index
merged_si_assm.reset_index(drop=True).head()

,code_module,code_presentation,id_student,region,imd_band,age_band,gender,highest_education,disability,final_result,id_assessment,assessment_type,date_submitted,date,weight,score
0,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,1752,TMA,18.0,19.0,10.0,78.0
1,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,1753,TMA,53.0,54.0,20.0,85.0
2,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,1754,TMA,115.0,117.0,20.0,80.0
3,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,1755,TMA,164.0,166.0,20.0,85.0
4,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,1756,TMA,212.0,215.0,30.0,82.0


In [18]:
cleaned_assessments = merged_si_assm

---

## Testing Area

In [19]:
pd.DataFrame(student_info.loc[student_info['id_student'] == 65002, ['final_result']])

,final_result
15,Withdrawn
395,Fail


In [20]:
pd.DataFrame(merged_assessments.loc[merged_assessments['id_student'] == 65002, ['id_assessment','score']])

,id_assessment,score
14,1752,66.0
515,1753,68.0
1768,1758,66.0
1982,1759,68.0


In [21]:
new_df = pd.DataFrame(columns=merged_assessments.columns)

In [22]:
for i, r in assessments['module_presentation'].iteritems():
    print(r)

KeyError: 'module_presentation'

In [ ]:
cleaned_assessments.iloc[0]

In [ ]:
cleaned_assessments.head()

In [ ]:
assessments.head()

In [ ]:
pd.DataFrame(cleaned_assessments.loc[0:3])

In [42]:
cleaned_assessments['code_module'].values[0]

'AAA'

In [47]:
missing_exams_list = list(missing_exams['id_assessment'])
count = 0
my_list = []
value_list =[]
cleaned_assessments['module_presentation'] = cleaned_assessments['code_module'] + cleaned_assessments['code_presentation']
assessments['module_presentation'] = assessments['code_module'] + assessments['code_presentation']


for i in cleaned_assessments['module_presentation'].values:
    for j in assessments['module_presentation'].values:
        if row == r:
            tests_student_has = list(cleaned_assessments.loc[cleaned_assessments['id_student'] == cleaned_assessments['id_student'][index], 'id_assessment'])
            bad_lists = tests_student_has+missing_exams_list
            if assessments['id_assessment'].values[i] not in bad_lists:
                value_list = [cleaned_assessments['code_module'].values[index], cleaned_assessments['code_presentation'].values[index], cleaned_assessments['id_student'].values[index], assessments['id_assessment'].values[i], cleaned_assessments['score'].values[index], cleaned_assessments['date_submitted'].values[index], assessments['assessment_type'].values[i], assessments['date'].values[i], assessments['weight'].values[i]]
                my_list.append(value_list)
                print(f"{len(my_list)}/infinite rows appended", end="\r")


AttributeError: 'numpy.ndarray' object has no attribute 'iteritems'

In [33]:
missing_exams_list = list(missing_exams['id_assessment'])
count = 0
my_list = []
value_list =[]
cleaned_assessments['module_presentation'] = cleaned_assessments['code_module'] + cleaned_assessments['code_presentation']
assessments['module_presentation'] = assessments['code_module'] + assessments['code_presentation']


for index, row in cleaned_assessments['module_presentation'].iteritems():
    for i, r in assessments['module_presentation'].iteritems():
        if row == r:
            tests_student_has = list(cleaned_assessments.loc[cleaned_assessments['id_student'] == cleaned_assessments['id_student'][index], 'id_assessment'])
            bad_lists = tests_student_has+missing_exams_list
            if assessments['id_assessment'][i] not in bad_lists:
                value_list = [cleaned_assessments['code_module'][index], cleaned_assessments['code_presentation'][index], cleaned_assessments['id_student'][index], assessments['id_assessment'][i], cleaned_assessments['score'][index], cleaned_assessments['date_submitted'][index], assessments['assessment_type'][i], assessments['date'][i], assessments['weight'][i]]
                my_list.append(value_list)
    print(f"{len(my_list)}/infinite rows appended", end="\r")
    

KeyboardInterrupt: 

In [32]:
pd.DataFrame(my_list)

,0,1,2,3,4,5,6,7,8
0,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN
1,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN
2,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN
3,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN
4,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...
569,AAA,2013J,74372.0,1756.0,49.0,166.0,TMA,215.0,30.0
570,AAA,2013J,74372.0,1756.0,49.0,166.0,TMA,215.0,30.0
571,AAA,2013J,74372.0,1756.0,49.0,166.0,TMA,215.0,30.0
572,AAA,2013J,74372.0,1756.0,49.0,166.0,TMA,215.0,30.0


In [168]:
missing_exams_list = list(missing_exams['id_assessment'])
couant = 0
my_list = []

cleaned_assessments['module_presentation'] = cleaned_assessments['code_module'] + cleaned_assessments['code_presentation']
assessments['module_presentation'] = assessments['code_module'] + assessments['code_presentation']

print(start)

for index, row in cleaned_assessments['module_presentation'].iteritems():
    for i, r in assessments['module_presentation'].iteritems():
        if row == r:
            tests_student_has = list(cleaned_assessments.loc[cleaned_assessments['id_student'] == cleaned_assessments['id_student'][index], 'id_assessment'])
            bad_lists = tests_student_has+missing_exams_list
            if assessments['id_assessment'][i] not in bad_lists:
                value_dict = {'code_module':cleaned_assessments['code_module'][index], 'code_presentation':cleaned_assessments['code_presentation'][index], 'id_student':cleaned_assessments['id_student'][index], 'id_assessment':assessments['id_assessment'][i], 'score':cleaned_assessments['score'][index], 'date_submitted':cleaned_assessments['date_submitted'][index],'assessment_type':assessments['assessment_type'][i], 'date':assessments['date'][i], 'weight':assessments['weight'][i]}
    my_list.append(value_dict)
    print(f"{len(my_list)}/infinite rows appended", end="\r")
print(time.process_time() - start)

894.671875


KeyboardInterrupt: 

In [79]:
#good

cleaned_assessments['module_presentation'] = cleaned_assessments['code_module'] + cleaned_assessments['code_presentation']
assessments['module_presentation'] = assessments['code_module'] + assessments['code_presentation']
start = time.process_time()
missing_exams_list = list(missing_exams['id_assessment'])
count = 0

for index, row in cleaned_assessments.iterrows():
    for i, r in assessments.iterrows():
        if assessments['module_presentation'][i] == cleaned_assessments['module_presentation'][index]:
            tests_student_has = list(cleaned_assessments.loc[cleaned_assessments['id_student'] == cleaned_assessments['id_student'][index], 'id_assessment'])
            bad_lists = tests_student_has+missing_exams_list
            if assessments['id_assessment'][i] not in bad_lists:
                value_series = pd.Series([cleaned_assessments['code_module'][index], cleaned_assessments['code_presentation'][index], cleaned_assessments['id_student'][index], assessments['id_assessment'][i], cleaned_assessments['score'][index], cleaned_assessments['date_submitted'][index], assessments['assessment_type'][i], assessments['date'][i], assessments['weight'][i]])
                new_df = new_df.append(value_series, ignore_index=True)
                print(f"{len(new_df)}/ rows appended", end="\r")
                continue

345.5625


KeyboardInterrupt: 

In [88]:
pd.DataFrame(my_list)[0:50]

,code_module,code_presentation,id_student,id_assessment,score,date_submitted,assessment_type,date,weight
0,AAA,2013J,11391,1756,78.0,18.0,TMA,215.0,30.0
1,AAA,2013J,11391,1756,85.0,53.0,TMA,215.0,30.0
2,AAA,2013J,11391,1756,80.0,115.0,TMA,215.0,30.0
3,AAA,2013J,11391,1756,85.0,164.0,TMA,215.0,30.0
4,AAA,2013J,11391,1756,82.0,212.0,TMA,215.0,30.0
5,AAA,2013J,28400,1756,70.0,22.0,TMA,215.0,30.0
6,AAA,2013J,28400,1756,68.0,52.0,TMA,215.0,30.0
7,AAA,2013J,28400,1756,70.0,121.0,TMA,215.0,30.0
8,AAA,2013J,28400,1756,64.0,164.0,TMA,215.0,30.0
9,AAA,2013J,28400,1756,60.0,212.0,TMA,215.0,30.0


In [10]:
start = time.process_time()
missing_exams_list = list(missing_exams['id_assessment'])
count = 0
my_list = []
d = {}
print(start)
for index, row in merged_assessments.iterrows():
    for i, r in assessments.iterrows():
        if assessments['code_module'][i] == merged_assessments['code_module'][index] and assessments['code_presentation'][i] == merged_assessments['code_presentation'][index]:
            value_dict = {'code_module':merged_assessments['code_module'][index], 'code_presentation':merged_assessments['code_presentation'][index], 'id_student':merged_assessments['id_student'][index], 'id_assessment':assessments['id_assessment'][i], 'score':merged_assessments['score'][index], 'date_submitted':merged_assessments['date_submitted'][index],'assessment_type':assessments['assessment_type'][i], 'date':assessments['date'][i], 'weight':assessments['weight'][i]}
            tests_student_has = list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment'])
            if assessments['id_assessment'][i] not in (tests_student_has):
                    if assessments['id_assessment'][i] not in (missing_exams_list):
                            d[count] = value_dict
                            count+=1
                            print(f"{len(d)}/ rows appended", end="\r")
                            continue
print(time.process_time() - start)

217.984375


KeyboardInterrupt: 

In [141]:
missing_exams_list

[1757,
 1763,
 14990,
 15002,
 15014,
 15025,
 40087,
 40088,
 30713,
 30718,
 30723,
 34872,
 34885,
 34898,
 34911,
 37424,
 37434,
 37444]

In [ ]:
start = time.process_time()
missing_exams_list = list(missing_exams['id_assessment'])
count = 0
my_list = []
d = {}
print(start)
for index, row in merged_assessments.iterrows():
    for i, r in assessments.iterrows():
        if assessments['code_module'][i] == merged_assessments['code_module'][index] and assessments['code_presentation'][i] == merged_assessments['code_presentation'][index]:
            value_dict = {'code_module':merged_assessments['code_module'][index], 'code_presentation':merged_assessments['code_presentation'][index], 'id_student':merged_assessments['id_student'][index], 'id_assessment':assessments['id_assessment'][i], 'score':merged_assessments['score'][index], 'date_submitted':merged_assessments['date_submitted'][index],'assessment_type':assessments['assessment_type'][i], 'date':assessments['date'][i], 'weight':assessments['weight'][i]}
            tests_student_has = list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment'])
            if assessments['id_assessment'][i] not in (tests_student_has):
                    if assessments['id_assessment'][i] not in (missing_exams_list):
                            d[count] = value_dict
                            count+=1
                            print(f"{len(d)}/ rows appended", end="\r")
                            continue
print(time.process_time() - start)

In [ ]:
d = {}
count = 0

start = time.process_time()
missing_exams_list = list(missing_exams['id_assessment'])
my_list = []
d = {}
print(start)
for index, row in merged_assessments.iterrows():
    for i, r in assessments.iterrows():
        if assessments['code_module'][i] == merged_assessments['code_module'][index] and assessments['code_presentation'][i] == merged_assessments['code_presentation'][index]:
            value_dict = {'code_module':merged_assessments['code_module'][index], 'code_presentation':merged_assessments['code_presentation'][index], 'id_student':merged_assessments['id_student'][index], 'id_assessment':assessments['id_assessment'][i], 'score':merged_assessments['score'][index], 'date_submitted':merged_assessments['date_submitted'][index],'assessment_type':assessments['assessment_type'][i], 'date':assessments['date'][i], 'weight':assessments['weight'][i]}
            tests_student_has = list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment'])
            if assessments['id_assessment'][i] not in (tests_student_has):
                    if assessments['id_assessment'][i] not in (missing_exams_list):
                            d[count] = value_dict
                            count+=1
                            print(f"{len(d)}/ rows appended", end="\r")
                            continue
print(time.process_time() - start)




In [ ]:
# the dictionary to pass to pandas dataframe
d = {}

# a counter to use to add entries to "dict"
i = 0 

# Example data to loop and append to a dataframe
data = [{"foo": "foo_val_1", "bar": "bar_val_1"}, 
       {"foo": "foo_val_2", "bar": "bar_val_2"}]

# the loop
for entry in data:

    # add a dictionary entry to the final dictionary
    d[i] = {"col_1_title": entry['foo'], "col_2_title": entry['bar']}
    
    # increment the counter
    i = i + 1

# create the dataframe using 'from_dict'
# important to set the 'orient' parameter to "index" to make the keys as rows
df = DataFrame.from_dict(d, "index")

In [86]:
missing_exams_list = list(missing_exams['id_assessment'])
count = 0
while count < 1:
    for index, row in merged_assessments.iterrows():
        for i, r in assessments.iterrows():
            if assessments['code_module'][i] == merged_assessments['code_module'][index]:
                 if assessments['code_presentation'][i] == merged_assessments['code_presentation'][index]:
                        value_dict = {'code_module':merged_assessments['code_module'][index], 'code_presentation':merged_assessments['code_presentation'][index], 'id_student':merged_assessments['id_student'][index], 'id_assessment':assessments['id_assessment'][i], 'score':merged_assessments['score'][index], 'date_submitted':merged_assessments['date_submitted'][index],'assessment_type':assessments['assessment_type'][i], 'date':assessments['date'][i], 'weight':assessments['weight'][i]}
                        tests_student_has = list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment'])
                        if assessments['id_assessment'][i] not in (tests_student_has):
                            if assessments['id_assessment'][i] not in (missing_exams_list):
                                print(assessments['id_assessment'][i], merged_assessments['id_student'][index])
                                continue

1754 65002.0
1755 65002.0
1756 65002.0
1756 74372.0
1753 94961.0
1754 94961.0
1755 94961.0
1756 94961.0
1755 106247.0
1756 106247.0
1753 111717.0
1755 111717.0
1755 129955.0
1756 129955.0
1754 135400.0
1755 135400.0
1756 135400.0
1753 141377.0
1754 141377.0
1755 141377.0
1756 141377.0
1756 147756.0
1753 147793.0
1754 147793.0
1755 147793.0
1756 147793.0
1755 148993.0
1756 148993.0
1754 155984.0
1755 155984.0
1756 155984.0
1754 159954.0
1755 159954.0
1756 159954.0
1753 181316.0
1754 181316.0
1755 181316.0
1756 181316.0
1753 185439.0
1754 185439.0
1755 185439.0
1756 185439.0
1753 188278.0
1754 188278.0
1755 188278.0
1756 188278.0
1755 202635.0
1756 202635.0
1755 205719.0
1756 205719.0
1755 231554.0
1753 235507.0
1754 235507.0
1755 235507.0
1756 235507.0
1755 260355.0
1756 260355.0
1753 268733.0
1755 268733.0
1756 268733.0
1755 277880.0
1756 277880.0
1755 302302.0
1756 302302.0
1753 305152.0
1754 305152.0
1755 305152.0
1756 305152.0
1756 307866.0
1753 331358.0
1754 331358.0
1755 331358.0


KeyboardInterrupt: 

In [53]:
count = 0
while count < 1:
    for index, row in merged_assessments.iterrows():
        for i, r in assessments.iterrows():
            if assessments['code_module'][i] == merged_assessments['code_module'][index]:
                 if assessments['code_presentation'][i] == merged_assessments['code_presentation'][index]:
                        value_dict = {'code_module':merged_assessments['code_module'][index], 'code_presentation':merged_assessments['code_presentation'][index], 'id_student':merged_assessments['id_student'][index], 'id_assessment':assessments['id_assessment'][i], 'score':merged_assessments['score'][index], 'date_submitted':merged_assessments['date_submitted'][index],'assessment_type':assessments['assessment_type'][i], 'date':assessments['date'][i], 'weight':assessments['weight'][i]}
                        if assessments['id_assessment'][i] not in list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment']):
                            new_df = new_df.append(value_dict, ignore_index=True)
                            print(f"{len(new_df)}/ rows appended", end="\r")
                            count += 1
                            continue

KeyboardInterrupt: 

In [32]:
for index, row in student_info.iterrows():
     for i, r in assessments.iterrows():
            if assessments['code_module'][i] == student_info['code_module'][index]:
                 if assessments['code_presentation'][i] == student_info['code_presentation'][index]:
                        if int(assessments['id_assessment'][i]) not in list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment']):
                            values = [student_info['code_module'][index], student_info['code_presentation'][index], student_info['id_student'][index], assessments['id_assessment'][i], assessments['assessment_type'][i], assessments['weight'][i]]
                            for m in list(merged_assessments.columns):
                                for n in values:
                                    new_df[]

SyntaxError: invalid syntax (Temp/ipykernel_130144/1292780854.py, line 9)

In [55]:
'''count = 0
while count < 1:
    for index, row in student_info.iterrows():
        for i, r in assessments.iterrows():
            if assessments['code_module'][i] == student_info['code_module'][index]:
                 if assessments['code_presentation'][i] == student_info['code_presentation'][index]:
                        if assessments['id_assessment'][i] not in list(merged_assessments.loc[merged_assessments['id_student'] == merged_assessments['id_student'][index], 'id_assessment']):
                            new_df = new_df.append([[student_info['code_module'][index], student_info['code_presentation'][index], student_info['id_student'][index], assessments['id_assessment'][i], assessments['assessment_type'][i], assessments['weight'][i]]], ignore_index=True)
                            print(f"{len(new_df)}/ rows appended", end="\r")
                            count += 1
                            continue'''

'count = 0\nwhile count < 1:\n    for index, row in student_info.iterrows():\n        for i, r in assessments.iterrows():\n            if assessments[\'code_module\'][i] == student_info[\'code_module\'][index]:\n                 if assessments[\'code_presentation\'][i] == student_info[\'code_presentation\'][index]:\n                        if assessments[\'id_assessment\'][i] not in list(merged_assessments.loc[merged_assessments[\'id_student\'] == merged_assessments[\'id_student\'][index], \'id_assessment\']):\n                            new_df = new_df.append([[student_info[\'code_module\'][index], student_info[\'code_presentation\'][index], student_info[\'id_student\'][index], assessments[\'id_assessment\'][i], assessments[\'assessment_type\'][i], assessments[\'weight\'][i]]], ignore_index=True)\n                            print(f"{len(new_df)}/ rows appended", end="\r")\n                            count += 1\n                            continue'

In [15]:
pd.DataFrame(merged_assessments.loc[merged_assessments['id_student'] == 11391, 'id_assessment'])

,id_assessment
0,1752
501,1753
842,1754
1032,1755
1457,1756


**Updated Dataframe**

**Size**

In [ ]:
md(f'''* Number of Rows: {len(merged_assessments)}
* Number of Columns: {len(merged_assessments.columns)}''')

**Data Types**

In [ ]:
merged_assessments.dtypes

* id_student and id_assessments are both categorical values and so should be converted to objects

In [ ]:
# converting the data types
merged_assessments = merged_assessments.astype({'id_assessment': int, 'id_student': int})
merged_assessments = merged_assessments.astype({'id_assessment': object, 'id_student': object})

**Null Values**

In [ ]:
# prints the sum of a columns null value
merged_assessments.isnull().sum()

* We have 2,873 null data points for assessment date. The documentation of this dataset states that if the exam date is missing then it is as the end of the last presentation week. We can find this information in the courses dataframe.

In [ ]:
# adding the dates for the null test dates
for index, row in merged_assessments[merged_assessments['date'].isna()].iterrows():
    merged_assessments.at[index, 'date'] = courses.loc[(courses['code_module'] == row['code_module']) & (courses['code_presentation'] == row['code_presentation']), 'module_presentation_length']

# reprinting to ensure it worked
merged_assessments.isnull().sum()

* There are 173 null values for score. These records are, unfortunately not of much interest to us, since score is what we are trying to find the relationship for, and so we will discard them. This leaves us with no null data in assessments.

In [ ]:
# removes any entry where the score is NaN
merged_assessments = merged_assessments.dropna(subset=['score'])

# reprinting to ensure it worked
merged_assessments.isnull().sum()

**Unique Counts**

In [ ]:
cleaned_assessments.nunique()

**Unique Categorical Values**

In [ ]:
unique_vals(cleaned_assessments)

**Duplicate Values:**

In [ ]:
duplicate_vals(cleaned_assessments)

**Statistics**

In [ ]:
cleaned_assessments.describe()

In [ ]:
cleaned_assessments

In [ ]:
assessments

In [ ]:
cleaned_assessments.loc[cleaned_assessments['id_student'] == 11391, 'id_assessment']

if a test id is in assessments in the same code module and presentation as a student is in:
    if the test is already in the dataframe under that student id:
        do nothing
    else:
        add the test with all the same student information, the assessment id, type, and weight to the dataframe

In [ ]:
count = 0

# iterate through cleaned_assessments dataframe
for i, r in assessments.iterrows():
    for index, row in cleaned_assessments.iterrows():
    # iterate through assessments dataframe
    
        # if the code module in cleaned_assessments is the same as the code_module in assessments
        # convert to strings to compare
        if str(cleaned_assessments['code_module'][index]) == str(assessments['code_module'][i]):
            # if the code presentations are also the same
            if str(cleaned_assessments['code_presentation'][index]) == str(assessments['code_presentation'][i]):
                # if the assessment id is not found under that student append another row with that students information and the test they are missing
                if assessments['id_assessment'][i] not in cleaned_assessments.loc[cleaned_assessments['id_student'] == cleaned_assessments['id_student'][index], 'id_assessment']:
                    cleaned_assessments = cleaned_assessments.append([cleaned_assessments['code_module'][index], cleaned_assessments['code_presentation'][index], cleaned_assessments['id_student'][index], cleaned_assessments['region'][index], cleaned_assessments['imd_band'][index], cleaned_assessments['age_band'][index], cleaned_assessments['gender'][index], cleaned_assessments['highest_education'][index], cleaned_assessments['disability'][index], assessments['id_assessment'][i], assessments['assessment_type'][i], assessments['weight'][i]])
                    count += 1
                    print(f"{count} rows appended", end="\r")

In [ ]:
aggregates = { 'assessment_type':'first','weight':'sum'}
assessments.groupby(['code_module','code_presentation']).aggregate(aggregates).reset_index()

In [ ]:

for index, row in cleaned_assessments.iterrows():
    for i, r in assessments.iterrows():
        if cleaned_assessments['code_module'][index] == assessments['code_module'][i]:
            if cleaned_assessments['code_presentation'][index] == assessments['code_presentation'][i]:
                cleaned_assessments.append([cleaned_assessments['code_module'][index], cleaned_assessments['code_presentation'][index], cleaned_assessments['id_student'][index], cleaned_assessments['region'][index], cleaned_assessments['imd_band'][index], cleaned_assessments['age_band'][index], cleaned_assessments['gender'][index], cleaned_assessments['highest_education'][index], cleaned_assessments['disability'][index], assessments['id_assessment'][i], assessments['assessment_type'][i], assessments['weight'][i]])
                
                

In [ ]:
for index, row in cleaned_assessments.loc[cleaned_assessments['final_result'] == 'Withdrawn'].iterrows():
    for i, r in assessments.iterrows():
        if cleaned_assessments['code_module'][index] == assessments['code_module'][i]:
            if cleaned_assessments['code_presentation'][index] == assessments['code_presentation'][i]:
                cleaned_assessments.append([cleaned_assessments['code_module'][index], cleaned_assessments['code_presentation'][index], cleaned_assessments['id_student'][index], cleaned_assessments['region'][index], cleaned_assessments['imd_band'][index], cleaned_assessments['age_band'][index], cleaned_assessments['gender'][index], cleaned_assessments['highest_education'][index], cleaned_assessments['disability'][index], assessments['id_assessment'][i], assessments['assessment_type'][i], assessments['weight'][i]])

In [ ]:
pd.concat(x for _, x in cleaned_assessments.groupby("id_assessment") if len(x) > 1).head()

In [ ]:
for i, r in assessments[student_info.loc['final_result'] == 'Withdrawn'].iterrows():
        cleaned_assessments.append(courses.loc[(courses['code_module'] == row['code_module']) & (courses['code_presentation'] == row['code_presentation']), 'module_presentation_length']

In [ ]:
student_assessment

In [ ]:
assessments

In [ ]:
cleaned_assessments

In [ ]:
for index, row in student_info.iterrows():
    for i, r in assessments.iterrows():
        if student_info['code_module'][index] == assessments['code_module'][i]:
            if student_info['code_presentation'][index] == assessments['code_presentation'][i]:
                new_df = new_df.append([student_info['code_module'][index], student_info['code_presentation'][index], student_info['id_student'][index], assessments['id_assessment'][i], assessments['assessment_type'][i], assessments['weight'][i]])
                print(f"{len(new_df)}/ rows appended", end="\r")